# Assignment 2
### Daniel Mehta

## Exercise 1: [POS (Part-of-Speech) Tagging with Hidden Markov Model](https://www.mygreatlearning.com/blog/pos-tagging/)

In [3]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [7]:
nltk.download('treebank')
nltk.download('universal_tagset')
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

[nltk_data] Downloading package treebank to
[nltk_data]     /Users/danielmehta/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/danielmehta/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [11]:
#split training and validation
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [13]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [15]:
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [17]:
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'VERB', 'PRT', 'NOUN', '.', 'X', 'ADJ', 'PRON', 'DET', 'CONJ', 'ADV', 'ADP', 'NUM'}


In [19]:
vocab = {word for word,tag in train_tagged_words}

In [23]:
#compute emission probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
     
    return (count_w_given_tag, count_tag)

In [25]:
# compute transition probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1]== t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [27]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[1.67955801e-01 3.06629837e-02 1.10589318e-01 3.48066315e-02
  2.15930015e-01 6.63904250e-02 3.55432779e-02 1.33609578e-01
  5.43278083e-03 8.38858187e-02 9.23572779e-02 2.28360966e-02]
 [4.01174158e-01 1.17416831e-03 2.50489235e-01 4.50097844e-02
  1.21330721e-02 8.29745606e-02 1.76125243e-02 1.01369865e-01
  2.34833662e-03 9.39334650e-03 1.95694715e-02 5.67514673e-02]
 [1.49133503e-01 4.39345129e-02 2.62344331e-01 2.40094051e-01
  2.88252197e-02 1.25838192e-02 4.65906132e-03 1.31063312e-02
  4.24540639e-02 1.68945398e-02 1.76826611e-01 9.14395228e-03]
 [8.96899477e-02 2.78940029e-03 2.18538776e-01 9.23720598e-02
  2.56410260e-02 4.61323895e-02 6.87694475e-02 1.72191828e-01
  6.00793920e-02 5.25694676e-02 9.29084867e-02 7.82104954e-02]
 [2.06419379e-01 1.85085520e-01 6.16951771e-02 1.60868734e-01
  7.57255405e-02 1.76821072e-02 5.41995019e-02 5.68902567e-02
  1.03786280e-02 2.57543717e-02 1.42225638e-01 3.07514891e-03]
 [1.14563107e-02 1.14563107e-02 6.96893215e-01 6.60194159e-02
  2

In [29]:
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,VERB,PRT,NOUN,.,X,ADJ,PRON,DET,CONJ,ADV,ADP,NUM
VERB,0.167956,0.030663,0.110589,0.034807,0.215930,0.066390,0.035543,0.133610,0.005433,0.083886,0.092357,0.022836
PRT,0.401174,0.001174,0.250489,0.045010,0.012133,0.082975,0.017613,0.101370,0.002348,0.009393,0.019569,0.056751
NOUN,0.149134,0.043935,0.262344,0.240094,0.028825,0.012584,0.004659,0.013106,0.042454,0.016895,0.176827,0.009144
.,0.089690,0.002789,0.218539,0.092372,0.025641,0.046132,0.068769,0.172192,0.060079,0.052569,0.092908,0.078210
X,0.206419,0.185086,0.061695,0.160869,0.075726,0.017682,0.054200,0.056890,0.010379,0.025754,0.142226,0.003075
ADJ,0.011456,0.011456,0.696893,0.066019,0.020971,0.063301,0.000194,0.005243,0.016893,0.005243,0.080583,0.021748
PRON,0.484738,0.014123,0.212756,0.041913,0.088383,0.070615,0.006834,0.009567,0.005011,0.036902,0.022323,0.006834
DET,0.040247,0.000287,0.635906,0.017393,0.045134,0.206411,0.003306,0.006037,0.000431,0.012074,0.009918,0.022855
CONJ,0.150384,0.004391,0.349067,0.035126,0.009330,0.113611,0.060373,0.123491,0.000549,0.057080,0.055982,0.040615
ADV,0.339022,0.014740,0.032196,0.139255,0.022886,0.130721,0.012025,0.071373,0.006982,0.081458,0.119472,0.029868


In [31]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and  probability states
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [33]:
random.seed(42)
rndom = [random.randint(1,len(test_set)) for x in range(10)]
test_run = [test_set[i] for i in rndom]
test_run_base = [tup for sent in test_run for tup in sent]
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [37]:
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  22.855857133865356
Viterbi Algorithm Accuracy:  95.95959595959596


In [ ]:
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words

start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

In [ ]:
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

rule_based_tagger = nltk.RegexpTagger(patterns)

In [ ]:
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and  probability states
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1]
        else:
            if state_max != 'X':
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)